In [23]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [6]:
def load_data(input_file='test.txt'):
    with open(input_file, 'r') as file:
        data = file.readlines()
    data = ''.join(data)
    return data

In [51]:
def parse_robot_data(input_text: str):
    """
    Parses the input text to extract blockades, boxes, robot position, and velocity sequence.

    Parameters:
        input_text (str): Multiline string containing the grid and velocity sequence separated by an empty line.

    Returns:
        blockades (np.ndarray): Array of complex numbers representing blockade positions.
        boxes (np.ndarray): Array of complex numbers representing box positions.
        robot (complex): Complex number representing the robot's position.
        velocities (np.ndarray): Array of complex numbers representing velocities.
    """
    # Split the input into lines
    lines = input_text.strip().split('\n')

    # Find the index of the empty line separating grid and velocities
    try:
        empty_line_index = lines.index('')
    except ValueError:
        raise ValueError("Input text must contain an empty line separating the grid and velocities.")

    # Split into grid and velocity lines
    grid_lines = lines[:empty_line_index]
    velocity_lines = lines[empty_line_index + 1:]

    # Initialize lists to store positions
    blockades_list = []
    boxes_list = []
    robot_position = None

    # Parse the grid
    for y, line in enumerate(grid_lines):
        for x, char in enumerate(line):
            coord = complex(x, y)
            if char == '#':
                blockades_list.append(coord)
            elif char == 'O':
                boxes_list.append(coord)
            elif char == '@':
                if robot_position is not None:
                    raise ValueError("Multiple robot positions found in the grid.")
                robot_position = coord
            # Ignore other characters (e.g., '.', etc.)

    if robot_position is None:
        raise ValueError("No robot position ('@') found in the grid.")

    # Convert lists to NumPy arrays
    blockades = np.array(blockades_list, dtype=complex)
    boxes = np.array(boxes_list, dtype=complex)
    robot = robot_position  # Single complex number

    # Parse the velocity sequence
    direction_map = {
        '>': 1 + 0j,   # Right
        '<': -1 + 0j,  # Left
        '^': 0 - 1j,   # Up
        'v': 0 + 1j    # Down
    }

    # Concatenate all velocity lines into a single string
    velocity_sequence = ''.join(velocity_lines).strip()

    # Convert each direction symbol to its corresponding complex velocity
    velocities_list = []
    for char in velocity_sequence:
        if char in direction_map:
            velocities_list.append(direction_map[char])
        else:
            # Ignore or handle invalid characters as needed
            print(f"Warning: Invalid direction symbol '{char}' encountered and skipped.")

    # Convert to NumPy array
    velocities = np.array(velocities_list, dtype=complex)

    return blockades, boxes, robot, velocities


In [26]:
data = load_data()
blockades, boxes, robot, velocities = parse_robot_data(data)

# get max and min of the blockades
max_blockades = np.max(blockades)
min_blockades = np.min(blockades)

max_blockades, min_blockades

((9+9j), 0j)

In [70]:
def plot_robot_grid(blockades: np.ndarray, boxes: np.ndarray, robot: complex, dim_start: complex, dim_end: complex):
    """
    Plots blockades, boxes, and robot positions on a grid using Plotly for interactivity.

    Parameters:
        blockades (np.ndarray): Array of complex numbers representing blockade positions.
        boxes (np.ndarray): Array of complex numbers representing box positions.
        robot (complex): Complex number representing the robot's position.
        dim_start (complex): Starting point defining plot's lower bounds.
        dim_end (complex): Ending point defining plot's upper bounds.
    """
    # Create trace for blockades
    trace_blockades = go.Scatter(
        x=blockades.real,
        y=blockades.imag,
        mode='markers',
        name='Blockades',
        marker=dict(
            symbol='square',      # Valid Plotly marker symbol
            size=12,
            color='black'
        )
    )

    # Create trace for boxes
    trace_boxes = go.Scatter(
        x=boxes.real,
        y=boxes.imag,
        mode='markers',
        name='Boxes',
        marker=dict(
            symbol='square',      # Valid Plotly marker symbol
            size=12,
            color='blue'
        )
    )

    # Create trace for robot
    trace_robot = go.Scatter(
        x=[robot.real],
        y=[robot.imag],
        mode='markers',
        name='Robot',
        marker=dict(
            symbol='star',        # Valid Plotly marker symbol
            size=16,
            color='red'
        )
    )

    # Define layout with specified dimensions
    layout = go.Layout(
        title='Robot Grid Visualization',
        xaxis=dict(
            title='Real Part',
            range=[dim_start.real - 1, dim_end.real + 1],
            zeroline=True
        ),
        yaxis=dict(
            title='Imaginary Part',
            range=[dim_end.imag + 1, dim_start.imag - 1],
            zeroline=True
        ),
        showlegend=True,
        width=700,
        height=700
    )

    # Create figure with all traces
    fig = go.Figure(data=[trace_blockades, trace_boxes, trace_robot], layout=layout)

    # Display the plot inline within the Jupyter Notebook
    iplot(fig)

In [53]:
plot_robot_grid(blockades, boxes, robot, min_blockades, max_blockades)

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

In [62]:
def apply_velocities(blockades: np.ndarray, boxes: np.ndarray, robot: complex, velocities: np.ndarray, dim_start: complex = 0+0j, dim_end: complex = 9+9j):
    """
    Applies a sequence of velocities to the robot and boxes on the grid without wrapping around.

    Parameters:
        blockades (np.ndarray): Array of complex numbers representing blockade positions.
        boxes (np.ndarray): Array of complex numbers representing box positions.
        robot (complex): Complex number representing the robot's current position.
        velocities (np.ndarray): Array of complex numbers representing the sequence of velocities.
        dim_start (complex): Starting point defining plot's lower bounds (default: 0+0j).
        dim_end (complex): Ending point defining plot's upper bounds (default: 9+9j).

    Returns:
        tuple:
            - Updated boxes as np.ndarray.
            - Updated robot position as complex.
    """
    # Define grid boundaries
    min_real, max_real = int(dim_start.real), int(dim_end.real)
    min_imag, max_imag = int(dim_start.imag), int(dim_end.imag)

    # Convert blockades and boxes to sets for faster lookup
    blockades_set = set(blockades)
    boxes_set = set(boxes)

    for vel in velocities:
        # Calculate the desired new position of the robot
        desired_robot_pos = robot + vel

        # Ensure positions are integers
        desired_robot_pos = complex(int(desired_robot_pos.real), int(desired_robot_pos.imag))

        # Check if the desired position is within grid boundaries
        if not (min_real <= desired_robot_pos.real <= max_real and min_imag <= desired_robot_pos.imag <= max_imag):
            # Desired position is outside the grid; skip this velocity
            print(f"Velocity {vel} skipped: Desired position {desired_robot_pos} is outside the grid boundaries.")
            continue

        if desired_robot_pos in blockades_set:
            # Blockade is directly in front; skip this velocity
            print(f"Velocity {vel} skipped: Blockade at {desired_robot_pos}")
            continue
        elif desired_robot_pos in boxes_set:
            # Attempt to push the box
            boxes_to_move = [desired_robot_pos]
            next_pos = desired_robot_pos + vel

            # Ensure next_pos has integer coordinates
            next_pos = complex(int(next_pos.real), int(next_pos.imag))

            # Check for consecutive boxes in the direction of velocity
            while next_pos in boxes_set:
                boxes_to_move.append(next_pos)
                next_pos += vel
                next_pos = complex(int(next_pos.real), int(next_pos.imag))

            # Check if the final position is free (not a blockade and within grid)
            if not (min_real <= next_pos.real <= max_real and min_imag <= next_pos.imag <= max_imag):
                # Final position is outside the grid; cannot push boxes
                print(f"Velocity {vel} skipped: Final position {next_pos} is outside the grid boundaries. Cannot push boxes {boxes_to_move}")
                continue
            if next_pos in blockades_set or next_pos in boxes_set:
                # Blockade or another box is at the end; cannot push boxes
                print(f"Velocity {vel} skipped: Blockade or box at {next_pos} prevents moving boxes {boxes_to_move}")
                continue
            else:
                # Move all boxes in boxes_to_move by velocity, from back to front
                for box in reversed(boxes_to_move):
                    new_box_pos = box + vel
                    new_box_pos = complex(int(new_box_pos.real), int(new_box_pos.imag))  # Ensure integer coordinates
                    boxes_set.remove(box)
                    boxes_set.add(new_box_pos)
                    print(f"Box moved from {box} to {new_box_pos}")

                # Move the robot to the desired position
                print(f"Robot moved from {robot} to {desired_robot_pos}")
                robot = desired_robot_pos
        else:
            # Move the robot freely
            print(f"Robot moved from {robot} to {desired_robot_pos}")
            robot = desired_robot_pos

    # Convert boxes_set back to np.ndarray
    updated_boxes = np.array(list(boxes_set))

    return updated_boxes, robot

In [63]:
updated_boxes, updated_robot = apply_velocities(blockades, boxes, robot, velocities, min_blockades, max_blockades)

Box moved from (3+4j) to (2+4j)
Robot moved from (4+4j) to (3+4j)
Robot moved from (3+4j) to (3+5j)
Robot moved from (3+5j) to (3+6j)
Box moved from (4+6j) to (5+6j)
Robot moved from (3+6j) to (4+6j)
Robot moved from (4+6j) to (4+5j)
Robot moved from (4+5j) to (3+5j)
Robot moved from (3+5j) to (3+6j)
Robot moved from (3+6j) to (3+5j)
Robot moved from (3+5j) to (4+5j)
Robot moved from (4+5j) to (4+6j)
Box moved from (5+6j) to (6+6j)
Robot moved from (4+6j) to (5+6j)
Box moved from (5+5j) to (5+4j)
Robot moved from (5+6j) to (5+5j)
Robot moved from (5+5j) to (5+6j)
Velocity 1j skipped: Blockade or box at (5+9j) prevents moving boxes [(5+7j), (5+8j)]
Robot moved from (5+6j) to (5+5j)
Robot moved from (5+5j) to (5+6j)
Box moved from (7+6j) to (8+6j)
Box moved from (6+6j) to (7+6j)
Robot moved from (5+6j) to (6+6j)
Robot moved from (6+6j) to (6+7j)
Box moved from (5+7j) to (4+7j)
Robot moved from (6+7j) to (5+7j)
Robot moved from (5+7j) to (6+7j)
Robot moved from (6+7j) to (6+8j)
Robot move

In [64]:
plot_robot_grid(blockades, updated_boxes, updated_robot, min_blockades, max_blockades)

##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########


In [65]:
def compute_gps_sum(boxes: np.ndarray):
    """
    Computes the sum of GPS coordinates for all boxes.

    Parameters:
        boxes (np.ndarray): Array of complex numbers representing box positions.

    Returns:
        int: Sum of GPS coordinates.
    """
    # Ensure all positions are integers
    boxes_int = np.array([complex(int(box.real), int(box.imag)) for box in boxes])
    gps_values = 100 * boxes_int.imag.astype(int) + boxes_int.real.astype(int)
    return np.sum(gps_values)


In [66]:
compute_gps_sum(updated_boxes)

10092

In [67]:
# real data
data = load_data('in.txt')
blockades, boxes, robot, velocities = parse_robot_data(data)

# get max and min of the blockades
max_blockades = np.max(blockades)
min_blockades = np.min(blockades)

max_blockades, min_blockades

((49+49j), 0j)

In [71]:
plot_robot_grid(blockades, boxes, robot, min_blockades, max_blockades)
##################################################
#OO..O.....O.OOO..O.....O....O.......O..O.....O.##
#.OO....O...O...#.O...O.......#.#O..O.OO.......###
#O.O.........OO.OOO.O...#...#O.#.OO.#..O.#O.O....#
#.O....OO..O.................#O.O.OO......O#OOO.O#
#O..#.O...O...O....OO..O.........O..O..O#O#..O..O#
#.#.OO.#....#..O#.....#..O....OO........#....O...#
#...O.O.O.......O.OOO...O.....OO...OO.....O..#..##
#O.#O..OO...OOO.O.#O...OOO.OO...O...O..O.........#
#...O........OO..O...###.OO.#O.OO.OO.............#
#..OOO...O..O.....O.#O.....OO.O..O...O...........#
#O..O#O.O.O.OO...#....O#.............#...#.OO.OO.#
#.#.OOOO...O..O.O..O...........#.....OOO.....O...#
#............O...O....O.....O#.OO...#.#..O...O#..#
#...O..O..#....#.OO..OO.#...O...OO..OO..O.##.#OO.#
#.O............#.........OO.#...OO....O........#.#
#...#..#.OO.O....O#........#OO.O#..O..OO..O.....O#
#.O.....O.....O..#..#OO.O...O........OO.OO.O.O.OO#
#.#O..O....O..O#.OOO.OOOOO..#.....OO....O...O.O..#
#...O..........#.......OO.O..O.#......#..O..OO...#
#O#.O.#...O......O#.OOO..O#.OOO.O.....O..OO......#
#.#O.O.#.....#.O......OO...O.O.O.OO.......O.O..OO#
#....O#....#OO....O.O#.OO.O....O.....O.O..OO.....#
#..O.O...#O..#O...OO.OO....O.....#....O..OO..O..O#
#.#OO.....O.OO..O.OO.O..@....#.OOOO.OO..O......OO#
#O..O...O...OO.O..O.....OOOO.....#O..#OOO.O..O..##
#..O.#OO#..O..#.......O..O.........O........OO#..#
#O..#O......#....OO...O.OOOO..#OOO..O...O....OOO.#
#..O.OO.......OO.......O...#.O..OO....OOO#..#.O..#
#...O...OOOO...#..O..O..O....O...#O....O..O.....O#
#.O....#..OOOO...OO..O.......O.O.O.....O.........#
#..OO.O.....OO..#.O...#O..O...O#.......O.O.OOO...#
#OO.OOO..O.......O....OO...OO#...#...OO....O..O.O#
#O.O..#O..#...O.O..O.O..O......OO#..O.......O...##
#.O#O.#..OOOO..OO......O...O..O.....O...O.O..O.#O#
#.....#OO.O.O....O.O.....O.O....O......OO..##.##O#
#O............O...OO.O.O....O..O..O#OO..O#..#..O.#
#..##OO.....O..#.O......O..#.O##..O..#OO....OO.O.#
#...O.#...O.....O#.O#O.O..OO##.....O#.....O.O..O.#
#O##..O.......OO....O...O....#O....O..O.O..O..O..#
#....#...O....O..O...........OO......#O..#.#O....#
#..OOO#.........#..........O....O.............O..#
#........OO.O...OO..O#OO..O#O......OOOO.O.#O..O..#
#.O.O.....O.O....#.O..OO....O#...O....OOO........#
#..O#.O....O..O..O#...O...O.....O...O.O.O..O.....#
##..O#.OO#.#..#..O...#...OO.....O.O#O.....OO..#..#
#...O.O.......#.OOOO.OO#..OO...OO.O.........OO...#
#..#.##O....O.........O.......OO......#O...O.OOO.#
#OO..O...OO....O.OO..O....#.#O...#O.O..OOOO.O.#O.#
##################################################

In [72]:
updated_boxes, updated_robot = apply_velocities(blockades, boxes, robot, velocities, min_blockades, max_blockades)

Robot moved from (24+24j) to (23+24j)
Robot moved from (23+24j) to (24+24j)
Box moved from (24+25j) to (24+26j)
Robot moved from (24+24j) to (24+25j)
Box moved from (24+27j) to (24+28j)
Box moved from (24+26j) to (24+27j)
Robot moved from (24+25j) to (24+26j)
Box moved from (24+29j) to (24+30j)
Box moved from (24+28j) to (24+29j)
Box moved from (24+27j) to (24+28j)
Robot moved from (24+26j) to (24+27j)
Box moved from (24+30j) to (24+31j)
Box moved from (24+29j) to (24+30j)
Box moved from (24+28j) to (24+29j)
Robot moved from (24+27j) to (24+28j)
Robot moved from (24+28j) to (25+28j)
Robot moved from (25+28j) to (25+29j)
Robot moved from (25+29j) to (25+30j)
Robot moved from (25+30j) to (26+30j)
Robot moved from (26+30j) to (26+29j)
Robot moved from (26+29j) to (27+29j)
Robot moved from (27+29j) to (26+29j)
Robot moved from (26+29j) to (26+28j)
Robot moved from (26+28j) to (26+29j)
Robot moved from (26+29j) to (26+28j)
Velocity (1+0j) skipped: Blockade at (27+28j)
Box moved from (26+27j

In [75]:
plot_robot_grid(blockades, updated_boxes, updated_robot, min_blockades, max_blockades)

In [74]:
compute_gps_sum(updated_boxes)

1517819

In [ ]:
def calculate_woman_salary(male_salary: int):
    """
    Calculates
    :param
    :param male_salary:
    :return:
    """

    return male_salary * 0.77